# 16.5 Case Study: A MongoDB JSON Document Database
### Installing the Python Libraries Required for Interacting with MongoDB
### keys.py 
## 16.5.1 Creating the MongoDB Atlas Cluster
### Creating Your First Database User
### Whitelist Your IP Address
### Connect to Your Cluster
## 16.5.2 Streaming Tweets into MongoDB
### Use Tweepy to Authenticate with Twitter

In [ ]:
import tweepy, keys

In [ ]:
auth = tweepy.OAuthHandler(
    keys.consumer_key, keys.consumer_secret)
auth.set_access_token(keys.access_token, 
    keys.access_token_secret)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)               

### Loading the Senators’ Data

In [ ]:
import pandas as pd

In [ ]:
senators_df = pd.read_csv('senators.csv')

In [ ]:
senators_df['TwitterID'] = senators_df['TwitterID'].astype(str)

In [ ]:
pd.options.display.max_columns = 6

In [ ]:
senators_df.head()

### Configuring the MongoClient 

In [ ]:
from pymongo import MongoClient

In [ ]:
atlas_client = MongoClient(keys.mongo_connection_string)

In [ ]:
db = atlas_client.senators 

### Setting up Tweet Stream

In [ ]:
from tweetlistener import TweetListener

In [ ]:
tweet_limit = 10000

In [ ]:
twitter_stream = tweepy.Stream(api.auth, 
    TweetListener(api, db, tweet_limit))

### Starting the Tweet Stream

In [ ]:
twitter_stream.filter(track=senators_df.TwitterHandle.tolist(),
    follow=senators_df.TwitterID.tolist())

### Class TweetListener

```python
# tweetlistener.py
"""TweetListener downloads tweets and stores them in MongoDB."""
import json
import tweepy

class TweetListener(tweepy.StreamListener):
    """Handles incoming Tweet stream."""

    def __init__(self, api, database, limit=10000):
        """Create instance variables for tracking number of tweets."""
        self.db = database
        self.tweet_count = 0
        self.TWEET_LIMIT = limit  # 10,000 by default
        super().__init__(api)  # call superclass's init

    def on_connect(self):
        """Called when your connection attempt is successful, enabling 
        you to perform appropriate application tasks at that point."""
        print('Successfully connected to Twitter\n')

    def on_data(self, data):
        """Called when Twitter pushes a new tweet to you."""
        self.tweet_count += 1  # track number of tweets processed
        json_data = json.loads(data)  # convert string to JSON
        self.db.tweets.insert_one(json_data)  # store in tweets collection
        print(f'    Screen name: {json_data["user"]["name"]}') 
        print(f'     Created at: {json_data["created_at"]}')         
        print(f'Tweets received: {self.tweet_count}')         

        # if TWEET_LIMIT is reached, return False to terminate streaming
        return self.tweet_count != self.TWEET_LIMIT
    
    def on_error(self, status):
        print(status)
        return True
```

### Counting Tweets for Each Senator

In [ ]:
db.tweets.create_index([('$**', 'text')])

In [ ]:
tweet_counts = []

In [ ]:
for senator in senators_df.TwitterHandle:
    tweet_counts.append(db.tweets.count_documents(
        {"$text": {"$search": senator}}))

### Show Tweet Counts for Each Senator 

In [ ]:
tweet_counts_df = senators_df.assign(Tweets=tweet_counts)

In [ ]:
tweet_counts_df.sort_values(by='Tweets', 
    ascending=False).head(10)

### Get the State Locations for Plotting Markers 

In [ ]:
from geopy import OpenMapQuest

In [ ]:
import time

In [ ]:
from state_codes import state_codes

In [ ]:
geo = OpenMapQuest(api_key=keys.mapquest_key) 

In [ ]:
states = tweet_counts_df.State.unique()

In [ ]:
states.sort()

In [ ]:
locations = []

In [ ]:
for state in states:
    processed = False
    delay = .1 
    while not processed:
        try:
            locations.append(
                geo.geocode(state_codes[state] + ', USA'))
            print(locations[-1])  
            processed = True
        except:  # timed out, so wait before trying again
            print('OpenMapQuest service timed out. Waiting.')
            time.sleep(delay)
            delay += .1

### Grouping the Tweet Counts by State 

In [ ]:
tweets_counts_by_state = tweet_counts_df.groupby(
    'State', as_index=False).sum()

In [ ]:
tweets_counts_by_state.head()

### Creating the Map 

In [ ]:
import folium

In [ ]:
usmap = folium.Map(location=[39.8283, -98.5795], 
                   zoom_start=4, detect_retina=True,
                   tiles='Stamen Toner')

### Creating a Choropleth to Color the Map 

In [ ]:
 choropleth = folium.Choropleth(
    geo_data='us-states.json',
    name='choropleth',
    data=tweets_counts_by_state,
    columns=['State', 'Tweets'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Tweets by State'
).add_to(usmap)

In [ ]:
layer = folium.LayerControl().add_to(usmap)

### Creating the Map Markers for Each State 

In [ ]:
sorted_df = tweet_counts_df.sort_values(
    by='Tweets', ascending=False)

In [ ]:
for index, (name, group) in enumerate(sorted_df.groupby('State')):
    strings = [state_codes[name]]  # used to assemble popup text

    for s in group.itertuples():
        strings.append(
            f'{s.Name} ({s.Party}); Tweets: {s.Tweets}')
        
    text = '<br>'.join(strings)  
    marker = folium.Marker(
        (locations[index].latitude, locations[index].longitude), 
        popup=text)
    marker.add_to(usmap) 

### Displaying the Map 

In [ ]:
usmap # in a notebook, this will display the map without the need to save it to a file 

If the preceding snippet does not display the map for you, uncomment the following snippet and execute it to save the HTML file to disk as we do in the chapter. You can then open that HTML file in your browser.

In [ ]:
# usmap.save('SenatorsTweets.html')

In [ ]:
##########################################################################
# (C) Copyright 2019 by Deitel & Associates, Inc. and                    #
# Pearson Education, Inc. All Rights Reserved.                           #
#                                                                        #
# DISCLAIMER: The authors and publisher of this book have used their     #
# best efforts in preparing the book. These efforts include the          #
# development, research, and testing of the theories and programs        #
# to determine their effectiveness. The authors and publisher make       #
# no warranty of any kind, expressed or implied, with regard to these    #
# programs or to the documentation contained in these books. The authors #
# and publisher shall not be liable in any event for incidental or       #
# consequential damages in connection with, or arising out of, the       #
# furnishing, performance, or use of these programs.                     #
##########################################################################